In [49]:
from pyspark.sql import SparkSession

In [50]:
spark = SparkSession.builder \
    .appName("JupyterStandalone") \
    .master("spark://8fa087ac675c:7077") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

In [51]:
sc = spark.sparkContext

In [52]:
# if you havent cached anything then it starts from beginning ie reading files, partitioning

In [53]:
rdd1=sc.parallelize([i for i in range(200)])

In [54]:
rdd2=rdd1.map(lambda x: x+1)
rdd3=rdd2.map(lambda x: x+1)
rdd4=rdd3.map(lambda x: x+1)
rdd5=rdd4.map(lambda x: x+1)
rdd6=rdd5.map(lambda x: x+1)
rdd7=rdd6.map(lambda x: x+1)

In [55]:
x=rdd7.collect()

In [56]:
x=rdd7.collect()

In [57]:
print(rdd7.toDebugString())

b'(2) PythonRDD[1] at collect at /tmp/ipykernel_770/584931559.py:1 []\n |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:297 []'


# vanna khoje ko when we do rdd7 .collect 2nd time as well it went back to create partions
so rdd7 back janxa janxa cached rdd vetyo vanae tei use garxa

if cached vete na vanae feri partition banauxa

rdd 7 sanga jo jo linked xa tyo matrai rdd calculate hunxa paxadi jada

# caching/persist concepts

tyo calculate vako partition memory mai store gari rakhu pare so that it doesnot go back every time to create partitions from the start

tyo partiitions haru store garnu paryo vanae cache persisit use garni

persist mempry only vanae ko nai cache ho

yedi memory only xa aani memory full xa vanae LRU(least recenty used) algorithm use hunxa to remove partitions and free space for the partitions to cache


we can do memory and disk ...

In [58]:
rdd2=rdd1.map(lambda x: x+1)
rdd3=rdd2.map(lambda x: x+1)
rdd4=rdd3.map(lambda x: x+1).cache()
rdd5=rdd4.map(lambda x: x+1)
rdd6=rdd5.map(lambda x: x+1)
rdd7=rdd6.map(lambda x: x+1)

In [59]:
x=rdd7.collect()

In [60]:
x=rdd7.collect()

In the **2nd run**, if you properly cached `rdd4` and the cache is still available, **Spark will use the cached data for `rdd4`** instead of recomputing from `rdd1`.

---

### How it works:

* On the **1st run**, since `rdd4` is not yet computed, Spark starts from `rdd1` and runs the full chain up to `rdd4`, caches it, then continues to `rdd7`.
* On the **2nd run**, Spark **checks the cache** for `rdd4`:

  * If the cached data exists and is not evicted, Spark **loads `rdd4` from cache**, skipping all upstream transformations from `rdd1` to `rdd3`.
  * Then it continues transformations from `rdd4` to `rdd7`.

---

### Important:

* The cache lives **only as long as your Spark context is alive and memory/disk space is sufficient**.
* If the cached data is evicted (e.g., due to memory pressure), Spark will recompute it again.
* If you call `.unpersist()` on `rdd4`, the cache is removed and Spark recomputes next time.

---

### TL;DR:

| Run     | Behavior                                        |
| ------- | ----------------------------------------------- |
| 1st run | Computes from `rdd1` → caches `rdd4` → finishes |
| 2nd run | Loads cached `rdd4` → computes rest             |

---

If you want, I can help you write a quick test to verify this caching behavior!


rdd4 ma action gare ko vayae bich tirai obviously tei cached nai use garxa

In [61]:
z=rdd6.collect()

In [62]:
z2=rdd6.collect()

In [63]:
z3=rdd6.collect()

In [64]:
z4=rdd6.collect()